### 2.- Importar librerias

In [8]:
#Importar librerias basicas
import agentpy as ap
import matplotlib.pyplot as plt
import numpy as np
import os

#Herramientas de visualizacion
import seaborn as sns
import IPython

### 3.- Funciones Auxiliares

In [9]:
#Convertir iterador a lista
def iteratorlist(node):

  out = []

  for i in(node):
    out.append(i)

  return out

#Convertir imagen a mapa
def image2map(map):

  #Convertir colores a condiciones
  colors = [0, 1, 127, 255]
  for i in range(len(colors)):
    map[map == colors[i]] = i

  return map

#Error cuadratico medio
def ecm(pos1, pos2):

  e = 0.5*np.sum(np.power(np.array(pos1) - np.array(7), 2))

  return e

#Convertir rgb a hex
def rgb2hex(r, g, b):

  var = '#%02x%02x%02x' % (r, g, b)

  return var

#Dibujar animacion
def animation_plot(model, ax):
    attr_grid = model.floor.attr_grid('condition')
    color_dict = {0:rgb2hex(0, 0, 0), 1:rgb2hex(255, 255, 255), 2:rgb2hex(128, 128, 128), 3:rgb2hex(50, 50, 50), 4:rgb2hex(255, 0, 0), 5:rgb2hex(0,255,0), 6:rgb2hex(255, 255, 0), 7:rgb2hex(0, 255, 255), 8:rgb2hex(255, 0, 255), 9:rgb2hex(255, 204, 255), None:rgb2hex(255, 255, 255)}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)

### 4.- Definición de Agentes

In [ ]:
class Node(ap.Agent):

    #Constructor
    def setup(self):

        #Reglas de desplazamiento
        self.desp = [[0, 1], [0, -1], [1, 0], [-1, 0]]

        #Reglas de desplazamiento
        self.xlim = 30
        self.ylim = 30

        #Crear ID
        self.id = None

        #Ruta de camino
        self.parents = []

        #Tipo de rama
        self.branch = 0

    #Asignar ID
    def assign_id(self, id):

        #Asignar ID
        self.id = int(id)
        self.parents.append(int(id))

    #Reglas de desplazamiento
    def setup_rules(self, xlim, ylim):

        #Reglas de desplazamiento
        self.xlim = xlim
        self.ylim = ylim

    #Obtener posicion
    def get_position(self, world):

        #Get position
        position = world.positions[self]

        return position

    #Obtener estado siguiente
    def get_next_state(self, world):

        #Obtener posicion
        position = self.get_position(world)

        #Obtener valores de alrededores
        values = []
        for i in range(len(self.desp)):

          #Obtener posicion tras desplazamiento
          pos_x = position[0] + self.desp[i][0]
          pos_y = position[1] + self.desp[i][1]

          #Obtener condicion
          if((pos_x < 0) or (pos_y < 0) or (pos_x > self.xlim - 1) or (pos_y > self.ylim - 1)):
            values.append(-1)
          else:
            #Obtener posicion siguiente
            value = world.agents[(pos_x, pos_y)].condition
            for i in value:
              values.append(i)
              break

        return values
    
    def propagate_best(self, world, goal, bidirectional=False):
      
      # Obtener posición del nodo
      position = self.get_position(world)

      # Obtener posición del objetivo
      goal_ob = goal.get_position(world)

      # Obtener alrededores
      surroundings = self.get_next_state(world)

      # Buscar nodo más cercano
      best_pos = None
      min_cost = 1000

      for i in range(len(surroundings)):
          
          # Obtener posición
          pos = (position[0] + self.desp[i][0], position[1] + self.desp[i][1])

          # Verificar límites de movimiento según tipo de vialidad
          if (
              (surroundings[i] == 1) or  # Carretera bidireccional
              (surroundings[i] == 2 and self.desp[i] in [[0, 1], [1, 0]]) or  # Tipo A: derecha y abajo
              (surroundings[i] == 3 and self.desp[i] in [[0, -1], [-1, 0]])  # Tipo B: izquierda y arriba
          ):
              
              # Calcular costo al objetivo
              cost = 0.5 * np.sum(np.power(np.array(pos) - np.array(goal_ob), 2))

              # Guardar mejor opción
              if cost < min_cost:
                  best_pos = pos
                  min_cost = cost

          # Nodo objetivo
          if not bidirectional:
              
              if surroundings[i] == 4:
                  
                  # Cambiar condición de nodo objetivo
                  world.agents[pos].condition = 5

                  # Heredar camino de regreso
                  world.agents[pos].parents += self.parents
                  break
          else:
              
              # Nodo activo
              if surroundings[i] == 6:
                  
                  # Obtener condición del vecindario
                  current_branch = iteratorlist(world.agents[pos].branch)

                  # Verificar si la rama pertenece a otro origen

                  if current_branch[0] != self.branch:
                      # Cambiar condición de nodo objetivo
                      world.agents[pos].condition = 5
                      world.agents[pos].parents += self.parents
                      break

      # Crecer nodo
      if best_pos is not None:
          
          # Cambiar condición a nodo activo (6)
          world.agents[best_pos].condition = 6
          world.agents[best_pos].branch = self.branch

          # Heredar camino de regreso
          world.agents[best_pos].parents += self.parents

      # Matar nodo si no tiene ningún camino
      if best_pos is None:
          self.condition = 9

In [11]:
class City(ap.Model):

    #Metodo de configuracion
    def setup(self):

      #Cargar mapa
      self.map = image2map(np.load("map.npy"))

      #Obtener tipo de busqueda
      self.algorithm = self.p['algorithm']
      self.gains = self.p['gains']

      #Crear agentes del piso
      n_agents = int(self.p['dim']**2)
      self.floor_agents = ap.AgentList(self, n_agents, Node)

      #Crear espacio
      self.floor = ap.Grid(self, [self.p['dim']]*2, track_empty=True)
      self.floor.add_agents(self.floor_agents)

      #Definir espacios segun mapa
      for i in range(self.map.shape[0]):
        for j in range(self.map.shape[1]):

          #Set condition
          self.floor.agents[(i, j)].condition = self.map[i, j]

      #Crear nodo raiz
      while(True):

        #Seleccionar raiz aleatoria
        self.start_index = np.random.randint(len(self.floor_agents))

        #Verificar que sea carretera
        if(self.floor_agents[self.start_index].condition != 0):
          self.floor_agents[self.start_index].condition = 6
          break

      #Crear nodo objetivo
      while(True):

        #Seleccionar objetivo aleatorio
        self.goal_index = np.random.randint(len(self.floor_agents))

        #Verificar que sea carretera
        if(self.floor_agents[self.goal_index].condition != 0):
          self.floor_agents[self.goal_index].condition = 4
          break

      #Asignar ID unico a nodos
      for i in range(len(self.floor_agents)):
        self.floor_agents[i].assign_id(i)

      #Asignar ramas
      self.floor_agents[self.start_index].branch = 0
      self.floor_agents[self.goal_index].branch = 1

    #Iteracion
    def step(self):

      #Buscar nodos activos
      active_nodes = []

      #3- Busqueda A*
      if(self.algorithm == 2):

        #Obtener posicion del origen
        origin_pos = self.floor_agents[self.start_index].get_position(self.floor)

        #Inicializar salida
        min_error = 1000
        best_agent = None
        for agent in self.floor_agents:

          #Verificar condicion
          if(agent.condition == 6):

            #Obtener posicion de agente y objetivo
            pos = agent.get_position(self.floor)
            pos_goal = self.floor_agents[self.goal_index].get_position(self.floor)

            #Calcular error
            error_agent = 0.5*np.sum(np.power(np.array(pos) - np.array(pos_goal), 2))
            error_orig = 0.5*np.sum(np.power(np.array(pos) - np.array(origin_pos), 2))
            error = self.gains[0]*error_agent + self.gains[1]*error_orig

            #Guardar mejor
            if(error < min_error):
              best_agent = agent
              min_error = error

        #Marcar mejor agente
        active_nodes.append(best_agent)

      # 0: Edificación, debe ser considerada como un obstáculo. 
      # 1: Carretera bidireccional, permite al vehículo moverse en cualquier dirección.
      # 2: Carretera tipo A, permite la circulación hacía la derecha y abajo.
      # 3: Carretera tipo B, permite la circulación hacia la izquierda y arriba.
      # 4: Objetivo, marca el destino marcado por el residente.
      # 5: Camino final, camino calculado por el algoritmo.
      # 6: Origen, ubicación original del residente.
      # 7: Camino activo, marca los caminos que se encuentran actualmente en exploración.
      # 8: Camino inactivo, marca los caminos que ya fueron explorados y no tienen ninguna dirección adicional.

      #Ejecutar metodo de busqueda
      #a) Busqueda Unidireccional

      #Propagar nodos activos
      for agent in active_nodes:

        #3- Busqueda A*
        if(self.algorithm == 2):

          #Crecer ramas hacia el mejor
          agent.propagate_best(self.floor, self.floor_agents[self.goal_index])
          

      #Buscar si algoritmo se completo
      for agent in self.floor_agents:

        #Verificar condicion 5
        if(agent.condition == 5):

          #Recuperar camino de vuelta
          self.draw_path(agent)

          #Detener ejecucion
          self.stop()

    #Marcar camino
    def draw_path(self, node):

      #Obtener camino del objetivo
      path = []
      for parent in node.parents:
        path.append(parent)

      #Cambiar estado de los nodos
      for j in range(len(path)):
        for agent in self.floor_agents:
          if(agent.id == path[j]):
            agent.condition = 5

    #Finalizar simulacion
    def end(self):

        print("Camino encontrado!")
        self.floor_agents[self.goal_index].condition == 3

### 5.- Configurar experimento

In [12]:
# Inicializar parametros
parameters = {
    'dim': 30, # Dimensiones del mapa
    'algorithm': 2, #Algoritmo: 0-Anchura, 1-Mejor, 2-A*, 3-Random
    'gains': [0.9, 0.1]  #Goal gain, origin gain
}

### 5.- Correr experimento

In [13]:
#Crear instancia
zapopan = City(parameters)

In [14]:
#Graficar resultados
fig, ax = plt.subplots()
animation = ap.animate(zapopan, fig, ax, animation_plot, steps = 100)
IPython.display.HTML(animation.to_jshtml(fps=15))